In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np

import csv
from tqdm import tqdm

In [2]:
DATA_DIR = "/Volumes/SSD970/"

TRAIN_SEG_DIR = os.path.join(DATA_DIR, "segmentations")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")

In [3]:
nii_files = glob.glob(TRAIN_SEG_DIR + "/*.nii")

In [27]:
header = [
    'id',
    'PatientID','slice_number',
    'C1','C2','C3','C4','C5','C6','C7',
    'min_row', 'max_row', 'min_column', 'max_column'
]

In [28]:
f = open('meta_segmentation.csv','w',newline='')
writer = csv.writer(f)
writer.writerow(header)

86

In [29]:
for path in tqdm(nii_files):
    seg_mask = nib.load(path).get_fdata()
    seg_mask = np.rot90(np.flip(seg_mask, axis=-1), axes=(0, 1))
    seg_mask[seg_mask > 7] = 0

    patient_id = path.split("/")[-1].split(".")[6]

    for slice in range(seg_mask.shape[2]):

        rows, columns = seg_mask[:, :, slice].nonzero()

        if len(rows) > 0:
            min_row, max_row, min_column, max_column = rows[0], rows[-1], np.min(columns), np.max(columns)
        else:
            min_row, max_row, min_column, max_column = 0,0,0,0

        row = [
            f"{patient_id}{slice+1:04d}",
            patient_id,(slice + 1),
            0,0,0,0,0,0,0,
            min_row, max_row, min_column, max_column
        ]

        segm_labels = np.unique(seg_mask[:, :, slice][rows, columns]).astype(int)

        for index in segm_labels:
            row[index + 2] = 1

        writer.writerow(row)

100%|██████████| 87/87 [02:36<00:00,  1.80s/it]


In [30]:
f.close()